In [4]:
# coding: utf-8
import warnings
warnings.filterwarnings('ignore')
import time
import os
import numpy as np
import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D,Activation
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
from collections import *
import pandas as pd

Using TensorFlow backend.


In [5]:
def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(word2idx[word])
            except:
                new_doc.append(0)
#         new_doc_arr = np.array(new_doc).reshape(1, max_doc_word_length)
        new_doc_arr = np.array(new_doc)
        new_corpus.append( new_doc_arr)
    return np.array(new_corpus)

In [6]:
# get data
# get texts data
category2idx = {'AllTogether': 0, 'Baseball': 1, 'Boy-Girl': 2, 'C_chat':  3, 'CVS': 4,
                  'GameSale': 5, 'GetMarry': 6, 'Lifeismoney': 7, 'LoL': 8, 'MH': 9, 'MLB': 10, 'Mobilecomm': 11, 
                'movie': 12,'MuscleBeach':  13, 'NBA': 14,  'SENIORHIGH': 15, 'Stock': 16, 
                'Tennis': 17, 'Tos': 18, 'WomenTalk': 19}

train_df_sample = pd.read_pickle('train.pkl').sample(frac=1, random_state=123)
train_texts = train_df_sample.values
label_list = train_df_sample.label

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values

train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])
    
texts_list = []
for text in train_texts_list:
    texts_list.append(text)
    
for text in test_texts:
    texts_list.append(text)

In [7]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec_20180430.model")
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)

vocab_num = len(wvv.items()) + 1

vocab_list = [(word, word_vectors[word]) for word, _ in wvv.items()]

In [8]:
del word_vectors, wvv, train_texts_list, answer

In [9]:
word_vec_len = 256
embedding_matrix = np.zeros((vocab_num , word_vec_len))
word2idx = {}

for i, vocab in enumerate(vocab_list):
    word, vec = vocab
    embedding_matrix[i + 1] = vec
    word2idx[word] = i + 1

In [10]:
embedding_layer = Embedding( input_dim= embedding_matrix.shape[0],output_dim= 256, weights=[embedding_matrix], 
                            input_length = 200,trainable=False)

In [11]:
max_doc_word_length = 200

x_trains_texts = train_df_sample.text.append(test_pickle_df.text)
X_train_texts = text_to_index(x_trains_texts)
X_train = pad_sequences(X_train_texts, maxlen= max_doc_word_length)

Y_label_list = np.zeros((36000, 2))
for ids in range(0, 36000):
    Y_label_list[ids][0] = label_list[ids][0]
    Y_label_list[ids][1] = label_list[ids][1]
print(Y_label_list.shape)

(36000, 2)


In [14]:
%%time
def getModel():
    model_temp = Sequential()
    model_temp.add(embedding_layer )
    model_temp.add(Conv1D(256, 3,
                    padding = 'same', 
                    )
             )
    model_temp.add(Flatten())
    model_temp.add(Dense(2, activation='relu'))
    model_temp.compile(optimizer='sgd',loss='mae')
    return model_temp

temp_res = []
test_sequences1 = X_train[36000:40000]
# X_train =sequences1
for i in range(0, 20):
    print("the " , i, " model training")
    model_tmp = getModel()
    left = i * 1800
    right = (i+1) * 1800
    history = model_tmp.fit(x=X_train[left:right], y = Y_label_list[left:right], batch_size= 1000, epochs = 150, verbose = 0)

    np_loss_history = np.array(history.history['loss'])
    np.savetxt("loss_history" + str(i) + ".txt", np_loss_history, delimiter=",")
    
    pre_res = model_tmp.predict(test_sequences1, batch_size= 1000, verbose=1)
    temp_res.append(pre_res)

the  0  model training
4000/4000 [==============================] - 6s 1ms/step
the  1  model training
4000/4000 [==============================] - 6s 1ms/step
the  2  model training
4000/4000 [==============================] - 6s 1ms/step
the  3  model training
4000/4000 [==============================] - 6s 2ms/step
the  4  model training
4000/4000 [==============================] - 6s 2ms/step
the  5  model training
4000/4000 [==============================] - 6s 1ms/step
the  6  model training
4000/4000 [==============================] - 6s 1ms/step
the  7  model training
4000/4000 [==============================] - 6s 1ms/step
the  8  model training
4000/4000 [==============================] - 6s 2ms/step
the  9  model training
4000/4000 [==============================] - 6s 2ms/step
the  10  model training
4000/4000 [==============================] - 6s 2ms/step
the  11  model training
4000/4000 [==============================] - 6s 1ms/step
the  12  model training
4000/4000 [===

In [57]:
final_res = []
for j in range(0, 4000):
    sum_a_res = 0
    for i in range(0, 20):
        sum_a_res += temp_res[i][j]
#         print(sum_a_res)
    mean_a_res = sum_a_res / 20
#     print(mean_a_res)
    final_res.append(mean_a_res)
print(len(final_res), final_res[0])

# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "server_result_mean_001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,good,bad" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(int(value[0])) + "," + str(int(value[1])) + '\n')
        ids += 1

4000 [9.842131   0.21455249]
4000


In [59]:
final_res = []
for j in range(0, 4000):
    print(j)
    min_sum_a_res = 1000
    temp_i = []
    for i in range(0, 20):
        chaju = 0
        initial = temp_res[i][j]
#         print(initial)
        sum_a_res = 0
        for m in range(0, 20):
#             print(initial, temp_res[m][j])
            sum_a_res += abs(initial - temp_res[m][j])
            
        temp_sum_a_res = sum_a_res[0] + sum_a_res[1]
        if min_sum_a_res > (temp_sum_a_res):
            print(min_sum_a_res, temp_sum_a_res)
            min_sum_a_res = temp_sum_a_res
            temp_i.append(i)
    
#     temp_i_sort = sorted(temp_i)
#     print(temp_i, )
    sum_a_res =0
    len_temp_i_sort = len(temp_i)
    if len_temp_i_sort > 3:
        len_temp_i_sort = 3
    for tis in temp_i_sort[0:len_temp_i_sort]:
#         print(temp_res[tis][j])
        sum_a_res += temp_res[tis][j]
    mean_a_res = sum_a_res / len_temp_i_sort
    final_res.append(mean_a_res)
#     print("\n\n", mean_a_res, "\n\n")


0
1000 119.20741
119.20741 117.37047
117.37047 116.350555
1
1000 176.55214
176.55214 162.35739
162.35739 162.13567
2
1000 181.0972
181.0972 146.07114
3
1000 168.73788
168.73788 155.3608
155.3608 145.34766
145.34766 144.69276
144.69276 142.285
4
1000 109.772606
109.772606 109.63539
109.63539 107.98262
107.98262 106.98709
5
1000 141.59206
141.59206 116.59855
6
1000 121.39272
121.39272 120.083405
120.083405 119.75153
7
1000 95.0776
8
1000 145.44441
145.44441 98.24573
98.24573 94.81593
94.81593 94.7975
9
1000 216.61456
216.61456 182.20177
182.20177 174.77039
174.77039 174.62851
10
1000 153.02846
11
1000 245.76695
245.76695 173.7472
173.7472 169.74202
169.74202 169.47736
169.47736 169.47694
169.47694 168.1747
12
1000 141.56908
141.56908 140.81631
13
1000 140.23032
140.23032 109.57802
14
1000 86.200264
86.200264 85.73667
15
1000 112.19501
112.19501 108.80424
108.80424 106.82473
16
1000 124.346176
124.346176 124.320366
17
1000 98.83468
98.83468 98.27228
98.27228 97.090004
18
1000 144.59724
14

155.82869 155.07063
155.07063 155.07062
161
1000 155.42305
155.42305 155.39395
155.39395 147.82149
147.82149 147.00696
162
1000 69.89562
69.89562 69.11675
69.11675 66.54234
66.54234 65.352066
163
1000 119.99373
119.99373 119.33917
119.33917 119.33914
164
1000 202.01163
202.01163 181.41919
165
1000 82.05733
82.05733 82.05732
166
1000 118.67465
118.67465 108.190285
108.190285 106.6797
106.6797 105.78517
167
1000 141.92415
141.92415 138.00562
168
1000 118.97496
118.97496 114.53542
169
1000 116.42916
116.42916 115.52965
170
1000 119.852356
171
1000 122.769
122.769 119.81185
119.81185 111.10806
111.10806 108.62194
172
1000 185.85657
185.85657 157.7358
157.7358 152.60632
152.60632 152.48358
152.48358 145.23221
173
1000 97.71526
97.71526 94.674934
174
1000 87.12304
87.12304 78.728645
78.728645 77.19309
77.19309 77.1457
175
1000 118.41439
118.41439 100.194916
100.194916 96.920975
176
1000 114.42001
114.42001 111.067566
177
1000 114.633026
114.633026 111.12012
111.12012 109.80884
178
1000 131.7

300
1000 135.11334
135.11334 127.38273
127.38273 125.23419
125.23419 123.59997
123.59997 121.99788
301
1000 109.94593
109.94593 106.433395
106.433395 106.008286
106.008286 105.86821
302
1000 137.14554
137.14554 125.71281
125.71281 124.63512
124.63512 122.12266
303
1000 166.18034
166.18034 134.67899
304
1000 152.576
152.576 150.25327
150.25327 148.36449
148.36449 146.31625
146.31625 146.29979
305
1000 121.6713
121.6713 121.46225
306
1000 117.51452
117.51452 109.45765
109.45765 107.79649
307
1000 159.56548
159.56548 137.68463
137.68463 129.99406
129.99406 123.52654
123.52654 116.86639
308
1000 166.643
166.643 134.26276
309
1000 174.22066
174.22066 129.81404
129.81404 129.79057
310
1000 157.9355
157.9355 126.214066
126.214066 126.01668
311
1000 157.74428
157.74428 157.21666
157.21666 157.09445
312
1000 124.74604
124.74604 121.54154
313
1000 131.86565
131.86565 131.80771
131.80771 130.58328
314
1000 136.39087
136.39087 130.31541
130.31541 129.14757
129.14757 125.78131
125.78131 125.05629
1

458
1000 203.72282
203.72282 190.56624
190.56624 190.56622
459
1000 167.1998
167.1998 165.53638
460
1000 219.71432
219.71432 192.14862
192.14862 191.67767
191.67767 190.58328
190.58328 189.49191
461
1000 256.14694
256.14694 214.5445
462
1000 261.471
261.471 193.384
463
1000 92.657005
92.657005 83.968124
83.968124 81.675896
464
1000 125.58262
465
1000 281.95703
281.95703 238.68303
238.68303 228.25418
228.25418 227.51433
466
1000 192.56906
192.56906 190.09943
190.09943 170.97852
170.97852 158.99237
158.99237 156.68912
156.68912 153.53822
467
1000 164.43744
164.43744 163.90018
163.90018 162.23575
162.23575 155.87103
468
1000 156.45346
156.45346 154.25629
154.25629 152.99521
152.99521 152.28224
469
1000 156.09201
156.09201 156.03285
156.03285 153.43355
153.43355 145.56592
145.56592 140.10423
470
1000 170.14691
170.14691 161.41412
471
1000 192.24457
472
1000 201.94066
201.94066 128.76047
128.76047 123.544754
123.544754 123.54475
473
1000 214.09183
214.09183 209.57639
474
1000 248.17958
248.

637
1000 176.05644
176.05644 143.86504
143.86504 143.86502
638
1000 173.5907
173.5907 148.87305
148.87305 148.76971
148.76971 148.2132
639
1000 130.36826
640
1000 208.53654
208.53654 178.66333
641
1000 258.46613
258.46613 245.16286
245.16286 199.59451
199.59451 198.38063
642
1000 176.99532
176.99532 167.0776
643
1000 147.10751
147.10751 143.6376
143.6376 143.03639
644
1000 173.09256
173.09256 170.67647
170.67647 170.0197
645
1000 222.05034
222.05034 181.91383
181.91383 181.8605
181.8605 181.81813
646
1000 128.40265
647
1000 162.8925
162.8925 161.41591
161.41591 154.88504
154.88504 151.65582
648
1000 160.60547
649
1000 193.96812
650
1000 203.64627
203.64627 188.19447
651
1000 194.33563
194.33563 166.52698
166.52698 153.77551
153.77551 153.59698
652
1000 143.7886
143.7886 141.43466
141.43466 141.0622
141.0622 139.84134
653
1000 140.81825
140.81825 129.86006
654
1000 170.84277
170.84277 165.11307
655
1000 126.413345
126.413345 123.51152
123.51152 111.134094
656
1000 203.51782
203.51782 17

...

215.8659 215.34738
3092
1000 177.60275
177.60275 172.41055
172.41055 172.24115
3093
1000 197.11084
197.11084 188.37985
188.37985 185.91183
185.91183 184.34567
184.34567 182.32257
3094
1000 200.23969
200.23969 185.70512
3095
1000 216.94681
216.94681 187.12616
3096
1000 305.90976
305.90976 233.5857
3097
1000 254.03522
254.03522 207.35612
3098
1000 164.47679
164.47679 164.47678
3099
1000 239.72273
239.72273 212.68611
212.68611 205.19287
205.19287 204.998
204.998 201.43663
3100
1000 199.66902
199.66902 196.86137
196.86137 192.92787
192.92787 192.15874
3101
1000 142.56563
142.56563 140.7161
140.7161 135.54236
3102
1000 170.35345
170.35345 167.34673
167.34673 165.17159
165.17159 161.59248
161.59248 160.71219
3103
1000 222.40149
222.40149 195.22101
3104
1000 167.63632
3105
1000 177.59973
177.59973 152.85663
3106
1000 221.25104
221.25104 175.63435
175.63435 175.28477
175.28477 173.26932
3107
1000 221.56367
221.56367 212.03363
3108
1000 177.64896
177.64896 170.1363
170.1363 164.83508
3109
1000 

1000 163.18784
163.18784 160.19579
3292
1000 167.50275
167.50275 153.37239
153.37239 153.19383
153.19383 152.40028
3293
1000 111.522255
3294
1000 179.96744
3295
1000 181.33524
181.33524 181.33522
3296
1000 199.9129
199.9129 189.87244
189.87244 183.52185
183.52185 181.88513
3297
1000 202.10445
202.10445 180.1503
3298
1000 146.9876
146.9876 139.78252
3299
1000 126.866486
126.866486 114.18613
114.18613 100.63805
3300
1000 187.97014
187.97014 182.6596
3301
1000 208.54732
208.54732 183.08595
183.08595 165.61417
3302
1000 221.80891
221.80891 197.50665
3303
1000 202.4515
202.4515 146.61281
146.61281 144.39574
144.39574 142.92288
3304
1000 167.59721
167.59721 155.14156
155.14156 153.94502
3305
1000 241.95612
241.95612 208.77092
208.77092 208.7709
3306
1000 204.83408
204.83408 173.86278
173.86278 163.73485
163.73485 160.14748
160.14748 158.07745
3307
1000 152.93436
152.93436 145.0948
3308
1000 127.00835
3309
1000 119.69061
119.69061 118.1439
3310
1000 174.56099
174.56099 169.819
169.819 157.514

1000 46.664852
46.664852 41.720688
3505
1000 226.41345
226.41345 189.48868
189.48868 188.48329
188.48329 185.50224
3506
1000 114.52889
114.52889 92.08523
92.08523 91.64448
3507
1000 46.664852
46.664852 41.720688
3508
1000 125.35251
125.35251 123.294815
123.294815 118.72863
3509
1000 135.56674
135.56674 135.45348
135.45348 134.10387
3510
1000 153.25305
153.25305 152.97694
3511
1000 162.09088
162.09088 122.12342
3512
1000 202.51488
202.51488 158.37498
158.37498 157.98653
3513
1000 96.81638
3514
1000 87.16205
87.16205 83.31988
83.31988 82.166336
82.166336 79.68408
3515
1000 109.61902
3516
1000 155.9147
155.9147 138.32233
3517
1000 140.26096
140.26096 121.35626
121.35626 121.356255
3518
1000 183.68422
183.68422 165.61523
165.61523 163.77763
163.77763 160.55447
160.55447 160.55446
3519
1000 104.16069
104.16069 96.478745
96.478745 92.58737
92.58737 89.98129
89.98129 89.804825
3520
1000 136.04436
136.04436 128.1252
128.1252 127.74673
3521
1000 123.67559
123.67559 101.15061
101.15061 101.00923

155.74083 155.26051
155.26051 155.2476
3653
1000 128.1201
128.1201 127.180695
3654
1000 184.96875
184.96875 147.70882
3655
1000 163.06953
163.06953 158.85234
158.85234 158.21143
3656
1000 130.67287
130.67287 124.70814
124.70814 122.95526
122.95526 122.95525
3657
1000 148.14253
148.14253 144.46536
144.46536 143.45282
3658
1000 215.70508
215.70508 160.31119
3659
1000 160.99483
160.99483 141.96991
141.96991 141.7732
141.7732 141.77318
3660
1000 202.16792
202.16792 164.44766
164.44766 159.32208
3661
1000 180.1759
180.1759 173.7813
173.7813 161.42857
161.42857 159.9614
159.9614 159.62798
3662
1000 147.17941
147.17941 143.1896
143.1896 141.21866
3663
1000 185.55186
185.55186 176.36996
176.36996 176.35007
3664
1000 234.05597
234.05597 205.1344
205.1344 203.62524
203.62524 187.13626
187.13626 181.2873
3665
1000 176.47766
176.47766 162.74576
3666
1000 251.02953
251.02953 201.15096
201.15096 198.21779
198.21779 196.24634
3667
1000 166.88841
3668
1000 120.3208
120.3208 119.3306
119.3306 117.97933

133.69757 133.69756
3858
1000 173.81888
173.81888 162.95438
3859
1000 89.02809
89.02809 88.715866
88.715866 85.29994
85.29994 84.876205
84.876205 84.87619
3860
1000 165.28886
165.28886 162.05708
162.05708 157.51085
157.51085 157.51083
3861
1000 142.41586
142.41586 139.37564
139.37564 138.6972
3862
1000 154.10234
154.10234 153.7527
153.7527 151.56471
151.56471 150.80501
3863
1000 140.2083
140.2083 139.541
3864
1000 201.86783
201.86783 197.53632
197.53632 178.38182
178.38182 176.7092
3865
1000 191.92152
191.92152 191.92151
3866
1000 178.00797
178.00797 163.02222
163.02222 159.78796
3867
1000 130.15083
130.15083 126.51395
126.51395 125.21796
125.21796 122.503235
3868
1000 101.31796
101.31796 100.87091
3869
1000 160.06503
160.06503 156.77515
3870
1000 235.8778
235.8778 162.43239
3871
1000 204.10872
204.10872 143.89091
143.89091 143.6665
3872
1000 270.58496
270.58496 195.58826
195.58826 195.50598
3873
1000 132.98178
132.98178 129.02756
129.02756 129.02754
3874
1000 201.9791
201.9791 176.433

In [54]:
print(len(final_res))
result_txt = "server_result_cluster_001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,good,bad" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(int(value[0])) + "," + str(int(value[1])) + '\n')
        ids += 1

4000


In [55]:
print(len(history.history["loss"]))

150


In [56]:
ss = [167, 5, 14,7]
s = sorted(ss)
print(ss, s[-3:])

[167, 5, 14, 7] [7, 14, 167]
